# Patient Summary Generator

This notebook generates a comprehensive patient summary from EMBR format data (optimized FHIR data) using OpenAI's GPT model.

## Process
1. Loads environment variables including OpenAI API key
2. Reads patient data from .embr files
3. Processes the data using a customized prompt
4. Generates a structured summary including:
   - Demographics
   - Conditions
   - Medications
   - Care plans
   - Procedures
   - Key encounters and observations

The summary maintains strict adherence to the provided data without inference or external recommendations.

In [25]:
import openai
from pathlib import Path
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [26]:
# Get first .embr file
embr_files = list(Path("../patients").glob("*.embr"))

# Arbitrary patient file, Adjust from 0-99 for different patients
with open(embr_files[11], 'r') as f:
    patient_data = f.read()

In [27]:

# Edit this prompt to fit your needs
prompt = f"""
You are analyzing patient data in EMBR format, an optimized summary of FHIR data. The data includes essential information.

Provide a summary with the following information:
Patient demographics
Conditions
Medications and MedicationRequests
Care plan and goals
Key procedures
Important Encounters and Observations


Only use the data provided, without adding, recommending, or inferring beyond it.
Reference any data with the dates and ID (e.g. EN-129 for Encounter 129).

Patient Data:
{patient_data}
"""

##### In this cell, we make an API call to OpenAI's GPT-4 model to analyze and summarize patient data. The model is given a system prompt identifying it as a healthcare professional, and a user prompt containing:
- The patient data in EMBR format
- Instructions to summarize specific aspects like demographics, conditions, medications etc.
- Guidelines to only use provided data and reference data points with dates/IDs

The response is stored in the 'summary' variable which contains the generated patient summary text.


In [32]:
response = openai.chat.completions.create(
    model="gpt-4o-mini", # Not all files can be summarized with gpt-4o, as the tokens are too high
    messages=[
        {"role": "system", "content": "You are a healthcare professional skilled at analyzing and summarizing patient records."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=5000
)

summary = response.choices[0].message.content
display(Markdown(summary))


### Patient Summary

**Patient Demographics:**
- **Name:** Mr. Chas Clark Green
- **Gender:** Male
- **Date of Birth:** October 30, 1985
- **Address:** 244 Mohr Club Unit 36, Hudson, MA, 01749
- **Marital Status:** Never Married
- **Communication Language:** English (United States)

---

**Conditions:**
- **Confirmed Conditions:**
  - Risk activity involvement (EN-2, 2001-12-12)
  - Medication review due (EN-8, 2003-12-24)
  - Prediabetes (EN-15, 2008-01-02)
  - Stress (EN-15, 2008-01-02, resolved 2017-01-11)
  - Anemia (EN-26, 2011-01-05)
  - Acute viral pharyngitis (EN-42, 2016-01-08)
  - Viral sinusitis (EN-49, 2016-10-19)
  - Obesity (BMI 30+) (EN-58, 2016-10-26)
  - Gingivitis (EN-58, 2016-10-26, resolved 2016-11-16)
  - Sleep disorder (EN-101, 2016-11-02)
  - Sleep apnea (EN-139, 2016-12-05)
  - Essential hypertension (EN-155, 2017-01-11)
  - Metabolic syndrome X (EN-549, 2022-02-09)
  - Injury of knee (EN-666, 2023-06-19)
  - Rupture of patellar tendon (EN-680, 2023-06-26)
  - Injury of medial collateral ligament of knee (EN-778, 2024-07-10)

---

**Medications and Medication Requests:**
- **Active Medication:**
  - Sodium fluoride 0.0272 MG/MG Oral Gel (MD-122, active)
- **Medication Requests (Stopped):**
  - Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet (MR-51, 2016-10-19)
  - Hydrochlorothiazide 25 MG Oral Tablet (multiple requests, last on MR-765, active as of 2024-02-21)
  - Ibuprofen 200 MG Oral Tablet (MR-459, 2020-05-19)
  - Naproxen sodium 220 MG Oral Tablet (MR-670, 2023-06-19; MR-782, 2024-07-10)
  - Meperidine Hydrochloride 50 MG Oral Tablet (MR-683, 2023-06-26)
  - Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet (MR-795, 2024-07-18)

---

**Care Plans and Goals:**
- **Diabetes Self-Management Plan:** 
  - Active since January 2, 2008 (CP-21).
- **Weight Reduction Regimen and CPAP Treatment for Sleep Apnea:**
  - Active since December 5, 2016 (CP-150).
- **Lifestyle Education Regarding Hypertension:** 
  - Active since January 11, 2017 (CP-208).
- **Wound Care for Laceration of Foot:**
  - Completed on May 19, 2020 (CP-463).
- **Physiotherapy Care Plan for Ruptured Patellar Tendon:**
  - Completed on June 19, 2023 (CP-674).
- **Physiotherapy Care Plan for Injury of Medial Collateral Ligament:**
  - Completed on July 10, 2024 (CP-786).

---

**Key Procedures:**
- **Sleep Apnea Assessments:**
  - Multiple assessments performed between November 2016 and February 2024.
- **Dental Procedures:**
  - Multiple dental consultations and care performed, including plaque removal and fluoride treatments from November 2016 to March 2023.
- **Emergency Room Admissions:**
  - Laceration treatment on May 19, 2020 (PR-458).
  - Knee injuries treated on June 19, 2023 (PR-669) and July 10, 2024 (PR-781).
- **Hospital Admissions for Joint Surgery:**
  - Non-urgent orthopedic admission for knee rupture on June 26, 2023 (PR-681).
  - Postoperative care for knee surgery on June 29, 2023 (PR-682).

---

**Important Encounters and Observations:**
- **Encounters:**
  - Regular general examinations and follow-ups with Dr. Rosa Caraballo and Dr. Araceli Willms from 2001 to 2024.
  
- **Observations:**
  - Vital signs and lab results consistently monitored, including weight, blood pressure, glucose levels, and hemoglobin A1c measurements at various encounters.
  - Notable blood pressure readings indicate hypertension with systolic values up to 151 mm(Hg) and diastolic values reaching 118 mm(Hg).

--- 

This summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.

# Analyzing Resource Usage

In the previous section, we generated a comprehensive patient summary by processing EMBR files and extracting relevant encounters and observations. Now, let's examine the resources and data processing steps required to build this summary. 

# Testability and Explainability
This approach ensures testability in your Generative AI results by verifying that the retrieved resources are accurate and free from hallucinations.

(This Code Will Update Obviously, Just Rough Implementation!)

In [89]:
def parse_embr_line(line):
    # Split on first space to separate ID from rest
    parts = line.split('=', 2)
    import re
    
    # Find all matches of key=value pattern
    matches = re.finditer(r'(\w+)=([^=]+?)(?=\s+\w+=|$)', line)
    
    # Convert matches to dictionary
    parsed = {}
    for match in matches:
        key = match.group(1)
        value = match.group(2).strip()
        parsed[key] = value
        
    return parsed

In [95]:
import re
from IPython.display import Markdown

# Find all instances of 2 letters followed by dash and numbers (e.g., EN-123, CO-456)
matches = re.findall(r'[A-Z]{2}-\d+', summary)

# Read the EMBR file and check first 10 chars for matches
with open(embr_files[11], 'r') as f:
    embr_content = f.read()
    embr_content = [line for line in embr_content.splitlines() if any(match in line for match in matches)]

formatted_lines = []

# Top-level heading for the summary
formatted_lines.append("**Resources In Summary:**")

for line in embr_content:
    result = parse_embr_line(line)

    # Add Observations with relevant information in indented bullet points
    formatted_lines.append("\n- **"+ ' '.join(line.split(' ')[0:2])+":**")
    if result.get('stat'):
        formatted_lines.append(f"  - Status: {result['stat']}")
    if result.get('date'):
        formatted_lines.append(f"  - Date: {result['date']}")
    if result.get('ref'):
        formatted_lines.append(f"  - References: {result['ref']}")
    
    # Additional details from other key-value pairs
    for key, value in result.items():
        if key not in ['ref', 'stat', 'date'] and value:
            # Format list-like values (like Codes) as comma-separated strings
            if isinstance(value, (list, set)):
                value = ', '.join(value)
            formatted_lines.append(f"  - {key.title()}: {value}")

# Add a concluding summary
formatted_lines.append("\nThis summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.")

# Join the formatted lines and display as markdown
display(Markdown('\n'.join(formatted_lines)))


**Resources In Summary:**

- **EN-2 Encounter:**
  - Status: finished
  - Date: 2001-12-12T15:08:17
  - References: {}
  - Codes: {Well child visit,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2001-12-12T15:23:17

- **EN-8 Encounter:**
  - Status: finished
  - Date: 2003-12-24T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2003-12-24T16:00:30

- **EN-15 Encounter:**
  - Status: finished
  - Date: 2008-01-02T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2008-01-02T15:57:52

- **EN-26 Encounter:**
  - Status: finished
  - Date: 2011-01-05T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2011-01-05T15:47:33

- **EN-42 Encounter:**
  - Status: finished
  - Date: 2016-01-08T02:08:17
  - References: {}
  - Codes: {Encounter for symptom,Acute viral pharyngitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-01-08T02:23:17

- **EN-49 Encounter:**
  - Status: finished
  - Date: 2016-10-19T04:08:17
  - References: {}
  - Codes: {Encounter for symptom,Viral sinusitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-10-19T04:23:17

- **EN-58 Encounter:**
  - Status: finished
  - Date: 2016-10-26T16:08:17
  - References: {}
  - Codes: {Encounter for check up,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-10-26T16:41:24

- **EN-101 Encounter:**
  - Status: finished
  - Date: 2016-11-02T16:08:17
  - References: {}
  - Codes: {Encounter for symptom,Sleep disorder,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-11-02T16:59:02

- **EN-139 Encounter:**
  - Status: finished
  - Date: 2016-12-04T15:56:45
  - References: {}
  - Codes: {Follow-up visit,Sleep disorder,v3-ActCode IMP}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2016-12-05T15:56:45

- **EN-155 Encounter:**
  - Status: finished
  - Date: 2017-01-11T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2017-01-11T16:05:32

- **EN-213 Encounter:**
  - Status: finished
  - Date: 2017-01-25T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Gingivitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-01-25T18:12:02

- **EN-227 Encounter:**
  - Status: finished
  - Date: 2017-02-10T15:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-02-10T15:23:17

- **EN-233 Encounter:**
  - Status: finished
  - Date: 2017-04-11T16:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-04-11T16:23:17

- **EN-239 Encounter:**
  - Status: finished
  - Date: 2017-06-10T16:08:17
  - References: {}
  - Codes: {Follow-up encounter,Essential hypertension,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2017-06-10T16:23:17

- **EN-245 Encounter:**
  - Status: finished
  - Date: 2018-01-17T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2018-01-17T16:03:57

- **EN-286 Encounter:**
  - Status: finished
  - Date: 2018-01-31T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Patient referral for dental care,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2018-01-31T18:47:19

- **EN-427 Encounter:**
  - Status: finished
  - Date: 2020-02-12T15:08:17
  - References: {}
  - Codes: {Encounter for check up,Gingivitis,ambulatory}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2020-02-12T22:23:49

- **EN-549 Encounter:**
  - Status: finished
  - Date: 2022-02-09T15:08:17
  - References: {}
  - Codes: {General examination of patient,ambulatory}
  - Provider: performer Dr. Rosa Caraballo
  - Facility: DR. HE'S PRIMARY CARE FOR ADULTS INC
  - Enddate: 2022-02-09T16:02:47

- **EN-666 Encounter:**
  - Status: finished
  - Date: 2023-06-19T18:43:43
  - References: {}
  - Codes: {Emergency room admission,Injury of knee,emergency}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2023-06-19T19:43:43

- **EN-680 Encounter:**
  - Status: finished
  - Date: 2023-06-26T18:43:43
  - References: {}
  - Codes: {Non-urgent orthopedic admission,Rupture of patellar tendon,v3-ActCode IMP}
  - Provider: performer Dr. Araceli Willms
  - Facility: MARLBOROUGH HOSPITAL CORP
  - Enddate: 2023-06-29T22:38:43

- **EN-778 Encounter:**
  - Status: finished
  - Date: 2024-07-10T22:38:43
  - References: {}
  - Codes: {Emergency room admission,Injury of knee,emergency}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2024-07-10T23:38:43

- **EN-802 Encounter:**
  - Status: finished
  - Date: 2024-07-20T16:13:43
  - References: {}
  - Codes: {Hospital admission,Patient transfer to skilled nursing facility,v3-ActCode IMP}
  - Provider: performer Dr. Gerry Davis
  - Facility: RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE
  - Enddate: 2024-07-29T16:28:43

- **CN-3 Condition:**
  - Status: confirmed
  - Date: 2001-12-12T16:17:32
  - References: {EN-2}
  - Codes: {Risk activity involvement}
  - Onsetdatetime: 2001-12-12T16:17:32

- **CN-9 Condition:**
  - Status: confirmed
  - Date: 2003-12-24T15:08:17
  - References: {EN-8}
  - Codes: {Medication review due}
  - Onsetdatetime: 2003-12-24T15:08:17
  - Abatementdatetime: 2016-10-26T16:08:17

- **CN-10 Condition:**
  - Status: confirmed
  - Date: 2003-12-24T16:00:30
  - References: {EN-8}
  - Codes: {Received higher education}
  - Onsetdatetime: 2003-12-24T16:00:30

- **CN-16 Condition:**
  - Status: confirmed
  - Date: 2008-01-02T15:08:17
  - References: {EN-15}
  - Codes: {Prediabetes}
  - Onsetdatetime: 2008-01-02T15:08:17

- **CN-17 Condition:**
  - Status: confirmed
  - Date: 2008-01-02T15:57:52
  - References: {EN-15}
  - Codes: {Stress}
  - Onsetdatetime: 2008-01-02T15:57:52
  - Abatementdatetime: 2017-01-11T16:05:32

- **CN-27 Condition:**
  - Status: confirmed
  - Date: 2011-01-05T15:08:17
  - References: {EN-26}
  - Codes: {Anemia}
  - Onsetdatetime: 2011-01-05T15:08:17

- **CN-43 Condition:**
  - Status: confirmed
  - Date: 2016-01-08T02:08:17
  - References: {EN-42}
  - Codes: {Acute viral pharyngitis}
  - Onsetdatetime: 2016-01-08T02:08:17
  - Abatementdatetime: 2016-01-21T17:08:17

- **CN-50 Condition:**
  - Status: confirmed
  - Date: 2016-10-19T04:08:17
  - References: {EN-49}
  - Codes: {Viral sinusitis}
  - Onsetdatetime: 2016-10-19T04:08:17
  - Abatementdatetime: 2016-10-31T04:08:17

- **CN-59 Condition:**
  - Status: confirmed
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Codes: {Body mass index 30+ - obesity}
  - Onsetdatetime: 2016-10-26T16:08:17

- **CN-60 Condition:**
  - Status: confirmed
  - Date: 2016-10-26T17:54:42
  - References: {EN-58}
  - Codes: {Gingivitis}
  - Onsetdatetime: 2016-10-26T17:54:42
  - Abatementdatetime: 2016-11-16T17:17:39

- **CN-102 Condition:**
  - Status: confirmed
  - Date: 2016-11-02T16:08:17
  - References: {EN-101}
  - Codes: {Sleep disorder}
  - Onsetdatetime: 2016-11-02T16:08:17

- **CN-140 Condition:**
  - Status: confirmed
  - Date: 2016-12-05T00:55:56
  - References: {EN-139}
  - Codes: {Sleep apnea}
  - Onsetdatetime: 2016-12-05T00:55:56

- **CN-156 Condition:**
  - Status: confirmed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Essential hypertension}
  - Onsetdatetime: 2017-01-11T15:08:17

- **CN-157 Condition:**
  - Status: confirmed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Medication review due}
  - Onsetdatetime: 2017-01-11T15:08:17
  - Abatementdatetime: 2019-01-23T15:08:17

- **CN-158 Condition:**
  - Status: confirmed
  - Date: 2017-01-11T16:05:32
  - References: {EN-155}
  - Codes: {Part-time employment}
  - Onsetdatetime: 2017-01-11T16:05:32
  - Abatementdatetime: 2018-01-17T16:03:57

- **CN-159 Condition:**
  - Status: confirmed
  - Date: 2017-01-11T16:05:32
  - References: {EN-155}
  - Codes: {Reports of violence in the environment}
  - Onsetdatetime: 2017-01-11T16:05:32
  - Abatementdatetime: 2018-01-17T16:03:57

- **CN-160 Condition:**
  - Status: confirmed
  - Date: 2017-01-11T17:44:13
  - References: {EN-155}
  - Codes: {Gingivitis}
  - Onsetdatetime: 2017-01-11T17:44:13
  - Abatementdatetime: 2017-01-25T18:07:02

- **CN-246 Condition:**
  - Status: confirmed
  - Date: 2018-01-17T16:03:57
  - References: {EN-245}
  - Codes: {Not in labor force}
  - Onsetdatetime: 2018-01-17T16:03:57
  - Abatementdatetime: 2019-01-23T16:00:18

- **CN-247 Condition:**
  - Status: confirmed
  - Date: 2018-01-17T16:03:57
  - References: {EN-245}
  - Codes: {Limited social contact}
  - Onsetdatetime: 2018-01-17T16:03:57
  - Abatementdatetime: 2019-01-23T16:00:18

- **CN-428 Condition:**
  - Status: confirmed
  - Date: 2020-02-12T17:51:08
  - References: {EN-427}
  - Codes: {Gingival disease}
  - Onsetdatetime: 2020-02-12T17:51:08
  - Abatementdatetime: 2020-02-12T20:04:49

- **CN-429 Condition:**
  - Status: confirmed
  - Date: 2020-02-12T20:59:51
  - References: {EN-427}
  - Codes: {Primary dental caries}
  - Onsetdatetime: 2020-02-12T20:59:51
  - Abatementdatetime: 2020-02-12T21:51:08

- **CN-550 Condition:**
  - Status: confirmed
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Codes: {Metabolic syndrome X}
  - Onsetdatetime: 2022-02-09T15:08:17

- **CN-667 Condition:**
  - Status: confirmed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Injury of knee}
  - Onsetdatetime: 2023-06-19T18:43:43
  - Abatementdatetime: 2023-09-07T22:38:43

- **CN-668 Condition:**
  - Status: confirmed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Rupture of patellar tendon}
  - Onsetdatetime: 2023-06-19T18:43:43
  - Abatementdatetime: 2023-09-07T22:38:43

- **CN-779 Condition:**
  - Status: confirmed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Injury of knee}
  - Onsetdatetime: 2024-07-10T22:38:43
  - Abatementdatetime: 2024-09-18T16:13:43

- **CN-780 Condition:**
  - Status: confirmed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Injury of medial collateral ligament of knee}
  - Onsetdatetime: 2024-07-10T22:38:43
  - Abatementdatetime: 2024-09-18T16:13:43

- **DR-93 DiagnosticReport:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58,OB-69,OB-70,OB-71,OB-72,OB-73,OB-74,OB-75,OB-76}
  - Codes: {Basic metabolic panel - Blood}
  - Category: ['v2-0074:LAB Laboratory']
  - Provider: MARLBOROUGH HOSPITAL CORP

- **DR-94 DiagnosticReport:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58,OB-77,OB-78,OB-79,OB-80}
  - Codes: {Lipid panel with direct LDL - Serum or Plasma}
  - Category: ['v2-0074:LAB Laboratory']
  - Provider: MARLBOROUGH HOSPITAL CORP

- **DR-95 DiagnosticReport:**
  - Status: final
  - Date: 2016-10-26T17:14:13
  - References: {EN-58,OB-83}
  - Codes: {Patient Health Questionnaire 2 item (Reported)}

- **DR-96 DiagnosticReport:**
  - Status: final
  - Date: 2016-10-26T17:54:42
  - References: {EN-58,OB-84}
  - Codes: {Drug Abuse Screening Test-10}

- **DR-203 DiagnosticReport:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,OB-169,OB-170,OB-171,OB-172,OB-173,OB-174,OB-175,OB-176}
  - Codes: {Basic metabolic panel - Blood}
  - Category: ['v2-0074:LAB Laboratory']
  - Provider: DR. HE'S PRIMARY CARE FOR ADULTS INC

- **DR-204 DiagnosticReport:**
  - Status: final
  - Date: 2017-01-11T16:31:10
  - References: {EN-155,OB-179}
  - Codes: {Generalized anxiety disorder 7 item}

- **DR-205 DiagnosticReport:**
  - Status: final
  - Date: 2017-01-11T17:03:41
  - References: {EN-155,OB-180}
  - Codes: {Patient Health Questionnaire 2 item (Reported)}

- **DR-206 DiagnosticReport:**
  - Status: final
  - Date: 2017-01-11T17:44:13
  - References: {EN-155,OB-181}
  - Codes: {Drug Abuse Screening Test-10}

- **DR-280 DiagnosticReport:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245,OB-256,OB-257,OB-258,OB-259,OB-260,OB-261,OB-262,OB-263}
  - Codes: {Basic metabolic panel - Blood}
  - Category: ['v2-0074:LAB Laboratory']
  - Provider: DR. HE'S PRIMARY CARE FOR ADULTS INC

- **DR-281 DiagnosticReport:**
  - Status: final
  - Date: 2018-01-17T16:40:07
  - References: {EN-245,OB-266}
  - Codes: {Drug Abuse Screening Test-10}

- **DR-588 DiagnosticReport:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549,OB-559,OB-560,OB-561,OB-562,OB-563,OB-564,OB-565,OB-566}
  - Codes: {Basic metabolic panel - Blood}
  - Category: ['v2-0074:LAB Laboratory']
  - Provider: DR. HE'S PRIMARY CARE FOR ADULTS INC

- **DR-589 DiagnosticReport:**
  - Status: final
  - Date: 2022-02-09T16:30:33
  - References: {EN-549,OB-569}
  - Codes: {Generalized anxiety disorder 7 item}

- **DR-590 DiagnosticReport:**
  - Status: final
  - Date: 2022-02-09T17:09:57
  - References: {EN-549,OB-570}
  - Codes: {Patient Health Questionnaire 2 item (Reported)}

- **DR-591 DiagnosticReport:**
  - Status: final
  - Date: 2022-02-09T17:30:12
  - References: {EN-549,OB-571}
  - Codes: {PHQ-9 quick depression assessment panel (Reported.PHQ)}

- **IS-304 ImagingStudy:**
  - Status: available
  - Date: 2018-01-31T17:24:14
  - References: {EN-286}
  - Codes: {Dental plain X-ray bitewing,Digital Radiography,Structure of region of internal part of mouth,Digital Intra-Oral X-Ray Image}

- **IS-675 ImagingStudy:**
  - Status: available
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Plain X-ray of knee region,Digital Radiography,Knee region structure,Image of knee}

- **IS-787 ImagingStudy:**
  - Status: available
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Plain X-ray of knee region,Digital Radiography,Knee region structure,Image of knee}

- **CP-21 CarePlan:**
  - Status: active
  - Date: 2008-01-02T15:08:17
  - References: {EN-15,CN-16}
  - Codes: {Diabetic diet at DR. HE'S PRIMARY CARE FOR ADULTS INC,Exercise therapy at DR. HE'S PRIMARY CARE FOR ADULTS INC,careplan-category assess-plan,Diabetes self management plan,Diabetic diet,Exercise therapy}
  - Text: Care Plan for Diabetes self management plan.Care plan is meant to treat Prediabetes.Activities: Diabetes self management planDiabetes self management plan
  - Intent: order

- **CP-150 CarePlan:**
  - Status: active
  - Date: 2016-12-05T00:55:56
  - References: {EN-139,CN-140}
  - Codes: {Weight reduction regimen at MARLBOROUGH HOSPITAL CORP,Warning. Avoid alcoholic drink (qualifier value) at MARLBOROUGH HOSPITAL CORP,Continuous positive airway pressure ventilation treatment at MARLBOROUGH HOSPITAL CORP,careplan-category assess-plan,Care plan,Weight reduction regimen,Warning. Avoid alcoholic drink (qualifier value),Continuous positive airway pressure ventilation treatment}
  - Text: Care Plan for Care plan.Care plan is meant to treat Sleep apnea.Activities: Care planCare planCare plan
  - Intent: order

- **CP-208 CarePlan:**
  - Status: active
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,CN-156}
  - Codes: {Prescribed activity/exercise education at DR. HE'S PRIMARY CARE FOR ADULTS INC,Counseling about alcohol consumption at DR. HE'S PRIMARY CARE FOR ADULTS INC,Dietary Approaches to Stop Hypertension diet at DR. HE'S PRIMARY CARE FOR ADULTS INC,Smoking cessation education at DR. HE'S PRIMARY CARE FOR ADULTS INC,careplan-category assess-plan,Lifestyle education regarding hypertension,Prescribed activity/exercise education,Counseling about alcohol consumption,Dietary Approaches to Stop Hypertension diet,Smoking cessation education}
  - Text: Care Plan for Lifestyle education regarding hypertension.Care plan is meant to treat Essential hypertension.Activities: Lifestyle education regarding hypertensionLifestyle education regarding hypertensionLifestyle education regarding hypertensionLifestyle education regarding hypertension
  - Intent: order

- **CP-463 CarePlan:**
  - Status: completed
  - Date: 2020-05-19T18:28:53
  - References: {EN-455,CN-457}
  - Codes: {Dressing change management at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Behavior to prevent infection (observable entity) at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Wound care,Dressing change management,Behavior to prevent infection (observable entity)}
  - Text: Care Plan for Wound care.Care plan is meant to treat Laceration of foot.Activities: Wound careWound care
  - Intent: order
  - Enddate: 2020-06-08T18:43:43

- **CP-674 CarePlan:**
  - Status: completed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666,CN-668}
  - Codes: {Rest, ice, compression and elevation treatment program at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Stretching exercises at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Physiotherapy care plan,Rest, ice, compression and elevation treatment program,Stretching exercises}
  - Text: Care Plan for Physiotherapy care plan.Care plan is meant to treat Rupture of patellar tendon.Activities: Physiotherapy care planPhysiotherapy care plan
  - Intent: order
  - Enddate: 2023-09-07T22:38:43

- **CP-786 CarePlan:**
  - Status: completed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778,CN-780}
  - Codes: {Rest, ice, compression and elevation treatment program at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,Stretching exercises at RESERVOIR CENTER FOR HEALTH & REHABILITATION, THE,careplan-category assess-plan,Physiotherapy care plan,Rest, ice, compression and elevation treatment program,Stretching exercises}
  - Text: Care Plan for Physiotherapy care plan.Care plan is meant to treat Injury of medial collateral ligament of knee.Activities: Physiotherapy care planPhysiotherapy care plan
  - Intent: order
  - Enddate: 2024-09-18T16:13:43

- **MR-51 MedicationRequest:**
  - Status: stopped
  - Date: 2016-10-19T04:08:17
  - References: {EN-49,CN-50}
  - Codes: {Amoxicillin 250 MG / Clavulanate 125 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms

- **MR-123 MedicationRequest:**
  - Status: stopped
  - Date: 2016-11-16T17:17:39
  - References: {EN-109,CN-60,MD-122}
  - Codes: {}
  - Intent: order
  - Requester: Dr. Araceli Willms

- **MR-195 MedicationRequest:**
  - Status: stopped
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-198 MedicationRequest:**
  - Status: stopped
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-276 MedicationRequest:**
  - Status: stopped
  - Date: 2018-01-17T15:08:17
  - References: {EN-245,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-300 MedicationRequest:**
  - Status: stopped
  - Date: 2018-01-31T18:11:49
  - References: {EN-286,MD-299}
  - Codes: {}
  - Intent: order
  - Requester: Dr. Araceli Willms
  - Reasoncode: ['sct:103697008 Patient referral for dental care']

- **MR-447 MedicationRequest:**
  - Status: stopped
  - Date: 2020-02-12T21:51:08
  - References: {EN-427,CN-60,MD-446}
  - Codes: {}
  - Intent: order
  - Requester: Dr. Araceli Willms

- **MR-459 MedicationRequest:**
  - Status: stopped
  - Date: 2020-05-19T18:43:43
  - References: {EN-455}
  - Codes: {Ibuprofen 200 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-584 MedicationRequest:**
  - Status: stopped
  - Date: 2022-02-09T15:08:17
  - References: {EN-549,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-670 MedicationRequest:**
  - Status: stopped
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Naproxen sodium 220 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-683 MedicationRequest:**
  - Status: stopped
  - Date: 2023-06-26T22:38:43
  - References: {EN-680}
  - Codes: {Meperidine Hydrochloride 50 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms
  - Dosageinstruction: 1 for 4.0 h

- **MR-765 MedicationRequest:**
  - Status: active
  - Date: 2024-02-21T15:08:17
  - References: {EN-721,CN-156}
  - Codes: {Hydrochlorothiazide 25 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Rosa Caraballo
  - Dosageinstruction: 1 for 1.0 d

- **MR-782 MedicationRequest:**
  - Status: stopped
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Naproxen sodium 220 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Gerry Davis
  - Dosageinstruction: Take as needed.

- **MR-795 MedicationRequest:**
  - Status: stopped
  - Date: 2024-07-18T02:13:43
  - References: {EN-792}
  - Codes: {Acetaminophen 325 MG / HYDROcodone Bitartrate 7.5 MG Oral Tablet}
  - Intent: order
  - Requester: Dr. Araceli Willms
  - Dosageinstruction: 1 for 6.0 h

- **PR-85 Procedure:**
  - Status: completed
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Codes: {Medication reconciliation}
  - Enddate: 2016-10-26T16:23:17

- **PR-86 Procedure:**
  - Status: completed
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Codes: {Assessment of health and social care needs}
  - Enddate: 2016-10-26T16:41:24

- **PR-87 Procedure:**
  - Status: completed
  - Date: 2016-10-26T16:41:24
  - References: {EN-58}
  - Codes: {Depression screening}
  - Enddate: 2016-10-26T16:53:19

- **PR-88 Procedure:**
  - Status: completed
  - Date: 2016-10-26T16:53:19
  - References: {EN-58}
  - Codes: {Depression screening}
  - Enddate: 2016-10-26T17:14:13

- **PR-89 Procedure:**
  - Status: completed
  - Date: 2016-10-26T17:14:13
  - References: {EN-58}
  - Codes: {Assessment of substance use}
  - Enddate: 2016-10-26T17:28:21

- **PR-90 Procedure:**
  - Status: completed
  - Date: 2016-10-26T17:28:21
  - References: {EN-58}
  - Codes: {Screening for drug abuse}
  - Enddate: 2016-10-26T17:54:42

- **PR-91 Procedure:**
  - Status: completed
  - Date: 2016-10-26T17:54:42
  - References: {EN-58,CN-60}
  - Codes: {Patient referral for dental care}
  - Enddate: 2016-10-26T17:59:42

- **PR-103 Procedure:**
  - Status: completed
  - Date: 2016-11-02T16:08:17
  - References: {EN-101,CN-102}
  - Codes: {Sleep apnea assessment}
  - Enddate: 2016-11-02T16:54:02

- **PR-104 Procedure:**
  - Status: completed
  - Date: 2016-11-02T16:54:02
  - References: {EN-101,CN-102}
  - Codes: {Referral to sleep apnea clinic}
  - Enddate: 2016-11-02T16:59:02

- **PR-141 Procedure:**
  - Status: completed
  - Date: 2016-12-04T15:56:45
  - References: {EN-139,CN-102}
  - Codes: {Sleep apnea assessment}
  - Enddate: 2016-12-04T16:50:56

- **PR-142 Procedure:**
  - Status: completed
  - Date: 2016-12-04T16:50:56
  - References: {EN-139}
  - Codes: {Continuous positive airway pressure titration}
  - Enddate: 2016-12-05T00:55:56

- **PR-182 Procedure:**
  - Status: completed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155,CN-102}
  - Codes: {Sleep apnea assessment}
  - Enddate: 2017-01-11T15:37:36

- **PR-183 Procedure:**
  - Status: completed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Assessment of health and social care needs}
  - Enddate: 2017-01-11T16:05:32

- **PR-184 Procedure:**
  - Status: completed
  - Date: 2017-01-11T16:05:32
  - References: {EN-155}
  - Codes: {Assessment of anxiety}
  - Enddate: 2017-01-11T16:31:10

- **PR-185 Procedure:**
  - Status: completed
  - Date: 2017-01-11T16:31:10
  - References: {EN-155}
  - Codes: {Depression screening}
  - Enddate: 2017-01-11T16:42:16

- **PR-186 Procedure:**
  - Status: completed
  - Date: 2017-01-11T16:42:16
  - References: {EN-155}
  - Codes: {Depression screening}
  - Enddate: 2017-01-11T17:03:41

- **PR-187 Procedure:**
  - Status: completed
  - Date: 2017-01-11T17:03:41
  - References: {EN-155}
  - Codes: {Assessment of substance use}
  - Enddate: 2017-01-11T17:16:53

- **PR-188 Procedure:**
  - Status: completed
  - Date: 2017-01-11T17:16:53
  - References: {EN-155}
  - Codes: {Screening for drug abuse}
  - Enddate: 2017-01-11T17:44:13

- **PR-189 Procedure:**
  - Status: completed
  - Date: 2017-01-11T17:44:13
  - References: {EN-155,CN-60}
  - Codes: {Patient referral for dental care}
  - Enddate: 2017-01-11T17:52:17

- **PR-214 Procedure:**
  - Status: completed
  - Date: 2017-01-25T15:08:17
  - References: {EN-213,CN-60}
  - Codes: {Dental consultation and report}
  - Enddate: 2017-01-25T15:39:21

- **PR-215 Procedure:**
  - Status: completed
  - Date: 2017-01-25T15:39:21
  - References: {EN-213,CN-60}
  - Codes: {Dental care}
  - Enddate: 2017-01-25T16:19:37

- **PR-216 Procedure:**
  - Status: completed
  - Date: 2017-01-25T16:19:37
  - References: {EN-213,CN-60}
  - Codes: {Removal of supragingival plaque and calculus from all teeth using dental instrument}
  - Enddate: 2017-01-25T16:41:34

- **PR-217 Procedure:**
  - Status: completed
  - Date: 2017-01-25T16:41:34
  - References: {EN-213,CN-60}
  - Codes: {Removal of subgingival plaque and calculus from all teeth using dental instrument}
  - Enddate: 2017-01-25T17:30:44

- **PR-218 Procedure:**
  - Status: completed
  - Date: 2017-01-25T17:30:44
  - References: {EN-213,CN-60}
  - Codes: {Examination of gingivae}
  - Enddate: 2017-01-25T18:07:02

- **PR-219 Procedure:**
  - Status: completed
  - Date: 2017-01-25T18:07:02
  - References: {EN-213,CN-60}
  - Codes: {Oral health education}
  - Enddate: 2017-01-25T18:12:02

- **PR-267 Procedure:**
  - Status: completed
  - Date: 2018-01-17T15:08:17
  - References: {EN-245,CN-102}
  - Codes: {Sleep apnea assessment}
  - Enddate: 2018-01-17T15:35:32

- **PR-268 Procedure:**
  - Status: completed
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Codes: {Assessment of health and social care needs}
  - Enddate: 2018-01-17T16:03:57

- **PR-269 Procedure:**
  - Status: completed
  - Date: 2018-01-17T16:03:57
  - References: {EN-245}
  - Codes: {Assessment of substance use}
  - Enddate: 2018-01-17T16:14:41

- **PR-270 Procedure:**
  - Status: completed
  - Date: 2018-01-17T16:14:41
  - References: {EN-245}
  - Codes: {Screening for drug abuse}
  - Enddate: 2018-01-17T16:40:07

- **PR-271 Procedure:**
  - Status: completed
  - Date: 2018-01-17T16:40:07
  - References: {EN-245}
  - Codes: {Patient referral for dental care}
  - Enddate: 2018-01-17T16:45:49

- **PR-287 Procedure:**
  - Status: completed
  - Date: 2018-01-31T15:08:17
  - References: {EN-286}
  - Codes: {Dental consultation and report}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T15:45:01

- **PR-288 Procedure:**
  - Status: completed
  - Date: 2018-01-31T15:45:01
  - References: {EN-286}
  - Codes: {Dental care}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T16:11:43

- **PR-289 Procedure:**
  - Status: completed
  - Date: 2018-01-31T16:11:43
  - References: {EN-286}
  - Codes: {Removal of supragingival plaque and calculus from all teeth using dental instrument}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T16:49:28

- **PR-290 Procedure:**
  - Status: completed
  - Date: 2018-01-31T16:49:28
  - References: {EN-286}
  - Codes: {Removal of subgingival plaque and calculus from all teeth using dental instrument}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T17:24:14

- **PR-291 Procedure:**
  - Status: completed
  - Date: 2018-01-31T17:24:14
  - References: {EN-286}
  - Codes: {Dental plain X-ray bitewing}
  - Enddate: 2018-01-31T17:54:14

- **PR-292 Procedure:**
  - Status: completed
  - Date: 2018-01-31T17:24:14
  - References: {EN-286}
  - Codes: {Examination of gingivae}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T18:11:49

- **PR-293 Procedure:**
  - Status: completed
  - Date: 2018-01-31T18:11:49
  - References: {EN-286}
  - Codes: {Dental fluoride treatment}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T18:34:39

- **PR-294 Procedure:**
  - Status: completed
  - Date: 2018-01-31T18:34:39
  - References: {EN-286}
  - Codes: {Oral health education}
  - Reasoncode: ['sct:103697008 Patient referral for dental care']
  - Enddate: 2018-01-31T18:47:19

- **PR-430 Procedure:**
  - Status: completed
  - Date: 2020-02-12T15:08:17
  - References: {EN-427,CN-60}
  - Codes: {Dental consultation and report}
  - Enddate: 2020-02-12T15:37:21

- **PR-431 Procedure:**
  - Status: completed
  - Date: 2020-02-12T15:37:21
  - References: {EN-427,CN-60}
  - Codes: {Dental care}
  - Enddate: 2020-02-12T16:01:12

- **PR-432 Procedure:**
  - Status: completed
  - Date: 2020-02-12T16:01:12
  - References: {EN-427,CN-60}
  - Codes: {Removal of supragingival plaque and calculus from all teeth using dental instrument}
  - Enddate: 2020-02-12T16:34:49

- **PR-433 Procedure:**
  - Status: completed
  - Date: 2020-02-12T16:34:49
  - References: {EN-427,CN-60}
  - Codes: {Removal of subgingival plaque and calculus from all teeth using dental instrument}
  - Enddate: 2020-02-12T17:00:42

- **PR-434 Procedure:**
  - Status: completed
  - Date: 2020-02-12T17:00:42
  - References: {EN-427,CN-60}
  - Codes: {Examination of gingivae}
  - Enddate: 2020-02-12T17:51:08

- **PR-435 Procedure:**
  - Status: completed
  - Date: 2020-02-12T17:51:08
  - References: {EN-427,CN-428}
  - Codes: {Dental surgical procedure}
  - Enddate: 2020-02-12T17:59:16

- **PR-436 Procedure:**
  - Status: completed
  - Date: 2020-02-12T17:59:16
  - References: {EN-427,CN-428}
  - Codes: {Dental application of desensitizing medicament}
  - Enddate: 2020-02-12T18:50:44

- **PR-437 Procedure:**
  - Status: completed
  - Date: 2020-02-12T18:50:44
  - References: {EN-427,CN-428}
  - Codes: {Gingivectomy or gingivoplasty, per tooth}
  - Enddate: 2020-02-12T20:04:49

- **PR-438 Procedure:**
  - Status: completed
  - Date: 2020-02-12T20:04:49
  - References: {EN-427}
  - Codes: {Postoperative care for dental procedure}
  - Enddate: 2020-02-12T20:59:51

- **PR-439 Procedure:**
  - Status: completed
  - Date: 2020-02-12T20:59:51
  - References: {EN-427,CN-429}
  - Codes: {Restoration of tooth with coverage of all cusps using dental filling material}
  - Enddate: 2020-02-12T21:51:08

- **PR-440 Procedure:**
  - Status: completed
  - Date: 2020-02-12T21:51:08
  - References: {EN-427,CN-60}
  - Codes: {Dental fluoride treatment}
  - Enddate: 2020-02-12T22:16:28

- **PR-441 Procedure:**
  - Status: completed
  - Date: 2020-02-12T22:16:28
  - References: {EN-427,CN-60}
  - Codes: {Oral health education}
  - Enddate: 2020-02-12T22:23:49

- **PR-458 Procedure:**
  - Status: completed
  - Date: 2020-05-19T18:28:53
  - References: {EN-455,CN-457}
  - Codes: {Suture open wound}
  - Enddate: 2020-05-19T18:43:43

- **PR-572 Procedure:**
  - Status: completed
  - Date: 2022-02-09T15:08:17
  - References: {EN-549,CN-102}
  - Codes: {Sleep apnea assessment}
  - Enddate: 2022-02-09T15:19:59

- **PR-573 Procedure:**
  - Status: completed
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Codes: {Assessment of health and social care needs}
  - Enddate: 2022-02-09T16:02:47

- **PR-574 Procedure:**
  - Status: completed
  - Date: 2022-02-09T16:02:47
  - References: {EN-549}
  - Codes: {Assessment of anxiety}
  - Enddate: 2022-02-09T16:30:33

- **PR-575 Procedure:**
  - Status: completed
  - Date: 2022-02-09T16:30:33
  - References: {EN-549}
  - Codes: {Depression screening}
  - Enddate: 2022-02-09T16:44:01

- **PR-576 Procedure:**
  - Status: completed
  - Date: 2022-02-09T16:44:01
  - References: {EN-549}
  - Codes: {Depression screening}
  - Enddate: 2022-02-09T17:09:57

- **PR-577 Procedure:**
  - Status: completed
  - Date: 2022-02-09T17:09:57
  - References: {EN-549}
  - Codes: {Depression screening using Patient Health Questionnaire Nine Item score}
  - Enddate: 2022-02-09T17:30:12

- **PR-669 Procedure:**
  - Status: completed
  - Date: 2023-06-19T18:43:43
  - References: {EN-666}
  - Codes: {Plain X-ray of knee region}
  - Enddate: 2023-06-19T19:13:43

- **PR-681 Procedure:**
  - Status: completed
  - Date: 2023-06-26T18:43:43
  - References: {EN-680,CN-668}
  - Codes: {Surgical manipulation of joint of knee}
  - Enddate: 2023-06-26T22:38:43

- **PR-682 Procedure:**
  - Status: completed
  - Date: 2023-06-26T22:38:43
  - References: {EN-680,CN-668}
  - Codes: {Postoperative care}
  - Enddate: 2023-06-29T22:38:43

- **PR-781 Procedure:**
  - Status: completed
  - Date: 2024-07-10T22:38:43
  - References: {EN-778}
  - Codes: {Plain X-ray of knee region}
  - Enddate: 2024-07-10T23:08:43

- **PR-803 Procedure:**
  - Status: completed
  - Date: 2024-07-20T16:13:43
  - References: {EN-802}
  - Codes: {History AND physical examination}
  - Enddate: 2024-07-20T16:28:43

- **PR-804 Procedure:**
  - Status: completed
  - Date: 2024-07-20T16:13:43
  - References: {EN-802}
  - Codes: {Initial patient assessment}
  - Enddate: 2024-07-20T16:28:43

- **PR-805 Procedure:**
  - Status: completed
  - Date: 2024-07-20T16:13:43
  - References: {EN-802}
  - Codes: {Development of individualized plan of care}
  - Enddate: 2024-07-20T16:28:43

- **PR-806 Procedure:**
  - Status: completed
  - Date: 2024-07-20T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-20T16:28:43

- **PR-807 Procedure:**
  - Status: completed
  - Date: 2024-07-20T16:13:43
  - References: {EN-802}
  - Codes: {Professional / ancillary services care}
  - Enddate: 2024-07-20T16:28:43

- **PR-808 Procedure:**
  - Status: completed
  - Date: 2024-07-21T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-21T16:28:43

- **PR-809 Procedure:**
  - Status: completed
  - Date: 2024-07-21T16:13:43
  - References: {EN-802}
  - Codes: {Physical therapy procedure}
  - Enddate: 2024-07-21T16:28:43

- **PR-810 Procedure:**
  - Status: completed
  - Date: 2024-07-21T16:13:43
  - References: {EN-802}
  - Codes: {Professional / ancillary services care}
  - Enddate: 2024-07-21T16:28:43

- **PR-811 Procedure:**
  - Status: completed
  - Date: 2024-07-22T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-22T16:28:43

- **PR-812 Procedure:**
  - Status: completed
  - Date: 2024-07-22T16:13:43
  - References: {EN-802}
  - Codes: {Physical therapy procedure}
  - Enddate: 2024-07-22T16:28:43

- **PR-813 Procedure:**
  - Status: completed
  - Date: 2024-07-23T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-23T16:28:43

- **PR-814 Procedure:**
  - Status: completed
  - Date: 2024-07-23T16:13:43
  - References: {EN-802}
  - Codes: {Professional / ancillary services care}
  - Enddate: 2024-07-23T16:28:43

- **PR-815 Procedure:**
  - Status: completed
  - Date: 2024-07-24T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-24T16:28:43

- **PR-816 Procedure:**
  - Status: completed
  - Date: 2024-07-25T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-25T16:28:43

- **PR-817 Procedure:**
  - Status: completed
  - Date: 2024-07-26T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-26T16:28:43

- **PR-818 Procedure:**
  - Status: completed
  - Date: 2024-07-27T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-27T16:28:43

- **PR-819 Procedure:**
  - Status: completed
  - Date: 2024-07-28T16:13:43
  - References: {EN-802}
  - Codes: {Nursing care/supplementary surveillance}
  - Enddate: 2024-07-28T16:28:43

- **PR-820 Procedure:**
  - Status: completed
  - Date: 2024-07-28T16:13:43
  - References: {EN-802}
  - Codes: {Physical therapy procedure}
  - Enddate: 2024-07-28T16:28:43

- **PR-821 Procedure:**
  - Status: completed
  - Date: 2024-07-29T16:13:43
  - References: {EN-802}
  - Codes: {Pre-discharge assessment}
  - Enddate: 2024-07-29T16:28:43

- **PR-822 Procedure:**
  - Status: completed
  - Date: 2024-07-29T16:13:43
  - References: {EN-802}
  - Codes: {Discharge from hospital}
  - Enddate: 2024-07-29T16:28:43

- **OB-44 Observation:**
  - Status: final
  - Date: 2016-01-08T02:08:17
  - References: {EN-42}
  - Codes: {Body temperature,Oral temperature}
  - Category: Vital
  - Valuequantity: 37.576 Cel

- **OB-61 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-62 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-63 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-64 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-65 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-66 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 71 mm(Hg),Systolic Blood Pressure 99 mm(Hg)}
  - Category: Vital

- **OB-67 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-68 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Vital

- **OB-69 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-70 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-71 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-72 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-73 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-74 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-75 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-76 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-77 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-78 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-79 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-80 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Laboratory

- **OB-81 Observation:**
  - Status: final
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Category: Social

- **OB-82 Observation:**
  - Status: final
  - Date: 2016-10-26T16:41:24
  - References: {EN-58}
  - Question: Do you consider yourself Hispanic/Latino?
  - Answer: No}
  - Category: Survey

- **OB-83 Observation:**
  - Status: final
  - Date: 2016-10-26T17:14:13
  - References: {EN-58}
  - Category: Survey

- **OB-84 Observation:**
  - Status: final
  - Date: 2016-10-26T17:54:42
  - References: {EN-58}
  - Category: Survey

- **OB-161 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-162 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-163 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-164 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-165 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-166 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 118 mm(Hg),Systolic Blood Pressure 151 mm(Hg)}
  - Category: Vital

- **OB-167 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-168 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Vital

- **OB-169 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-170 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-171 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-172 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-173 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-174 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-175 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-176 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Laboratory

- **OB-177 Observation:**
  - Status: final
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Category: Social

- **OB-178 Observation:**
  - Status: final
  - Date: 2017-01-11T16:05:32
  - References: {EN-155}
  - Question: Do you consider yourself Hispanic/Latino?
  - Answer: No}
  - Category: Survey

- **OB-179 Observation:**
  - Status: final
  - Date: 2017-01-11T16:31:10
  - References: {EN-155}
  - Category: Survey

- **OB-180 Observation:**
  - Status: final
  - Date: 2017-01-11T17:03:41
  - References: {EN-155}
  - Category: Survey

- **OB-181 Observation:**
  - Status: final
  - Date: 2017-01-11T17:44:13
  - References: {EN-155}
  - Category: Survey

- **OB-228 Observation:**
  - Status: final
  - Date: 2017-02-10T15:08:17
  - References: {EN-227}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 100 mm(Hg),Systolic Blood Pressure 131 mm(Hg)}
  - Category: Vital

- **OB-234 Observation:**
  - Status: final
  - Date: 2017-04-11T16:08:17
  - References: {EN-233}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 106 mm(Hg),Systolic Blood Pressure 126 mm(Hg)}
  - Category: Vital

- **OB-240 Observation:**
  - Status: final
  - Date: 2017-06-10T16:08:17
  - References: {EN-239}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 107 mm(Hg),Systolic Blood Pressure 133 mm(Hg)}
  - Category: Vital

- **OB-248 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-249 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-250 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-251 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-252 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-253 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 106 mm(Hg),Systolic Blood Pressure 125 mm(Hg)}
  - Category: Vital

- **OB-254 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-255 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Vital

- **OB-256 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-257 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-258 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-259 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-260 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-261 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-262 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-263 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Laboratory

- **OB-264 Observation:**
  - Status: final
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Category: Social

- **OB-265 Observation:**
  - Status: final
  - Date: 2018-01-17T16:03:57
  - References: {EN-245}
  - Question: Do you consider yourself Hispanic/Latino?
  - Answer: No}
  - Category: Survey

- **OB-266 Observation:**
  - Status: final
  - Date: 2018-01-17T16:40:07
  - References: {EN-245}
  - Category: Survey

- **OB-551 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-552 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-553 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-554 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-555 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-556 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Codes: {Blood pressure panel with all children optional,Diastolic Blood Pressure 108 mm(Hg),Systolic Blood Pressure 124 mm(Hg)}
  - Category: Vital

- **OB-557 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-558 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Vital

- **OB-559 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-560 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-561 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-562 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-563 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-564 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-565 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-566 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Laboratory

- **OB-567 Observation:**
  - Status: final
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Category: Social

- **OB-568 Observation:**
  - Status: final
  - Date: 2022-02-09T16:02:47
  - References: {EN-549}
  - Question: Do you consider yourself Hispanic/Latino?
  - Answer: No}
  - Category: Survey

- **OB-569 Observation:**
  - Status: final
  - Date: 2022-02-09T16:30:33
  - References: {EN-549}
  - Category: Survey

- **OB-570 Observation:**
  - Status: final
  - Date: 2022-02-09T17:09:57
  - References: {EN-549}
  - Category: Survey

- **OB-571 Observation:**
  - Status: final
  - Date: 2022-02-09T17:30:12
  - References: {EN-549}
  - Category: Survey

- **IM-92 Immunization:**
  - Status: completed
  - Date: 2016-10-26T16:08:17
  - References: {EN-58}
  - Codes: {Influenza, seasonal, injectable, preservative free}
  - Primarysource: True

- **IM-201 Immunization:**
  - Status: completed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Influenza, seasonal, injectable, preservative free}
  - Primarysource: True

- **IM-202 Immunization:**
  - Status: completed
  - Date: 2017-01-11T15:08:17
  - References: {EN-155}
  - Codes: {Td (adult), 5 Lf tetanus toxoid, preservative free, adsorbed}
  - Primarysource: True

- **IM-279 Immunization:**
  - Status: completed
  - Date: 2018-01-17T15:08:17
  - References: {EN-245}
  - Codes: {Influenza, seasonal, injectable, preservative free}
  - Primarysource: True

- **IM-587 Immunization:**
  - Status: completed
  - Date: 2022-02-09T15:08:17
  - References: {EN-549}
  - Codes: {Influenza, seasonal, injectable, preservative free}
  - Primarysource: True

- **MD-122 Medication:**
  - Status: active
  - References: {}
  - Codes: {sodium fluoride 0.0272 MG/MG Oral Gel}

- **MA-303 MedicationAdministration:**
  - Status: completed
  - Date: 2018-01-31T18:11:49
  - References: {EN-286}
  - Codes: {sodium fluoride 0.0272 MG/MG Oral Gel,Patient referral for dental care}

- **MA-450 MedicationAdministration:**
  - Status: completed
  - Date: 2020-02-12T21:51:08
  - References: {EN-427,CN-60}
  - Codes: {sodium fluoride 0.0272 MG/MG Oral Gel}

This summary reflects the patient's ongoing healthcare management, highlighting key health conditions, treatment plans, encounters, and significant laboratory observations.